In [ ]:
# Imports
import pandas as pd
#from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import torch
import pickle
import tensorflow as tf
from numpy import array
import numpy as np
from numpy import argmax
import torch.nn as nn
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



In [ ]:
#---Reference---
#https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456

In [ ]:
data = pd.read_table('hindi_dataset.tsv',nrows=100)
data




In [ ]:
torch.cuda.is_available()
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"

In [ ]:
data['text']= data['text'].str.lower()
data['text'] = data['text'].str.replace('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]','')

file1 = open("stopwords.txt", encoding="utf8") 
q=file1.read()
split1=q.split()

data['text'] = data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in split1]))

data.loc[data['task_1'] == "NOT", 'task_1'] = 0
data.loc[data['task_1'] == "HOF", 'task_1'] = 1

X = data.iloc[:, 1].values  
Y= data.iloc[:, 2].values  

Y = np.asarray(Y).astype('float32')

In [ ]:
#loading the embedding stored for task1
model_path = "task1.pth"
state_dict = torch.load(model_path)
embedding_weights=state_dict['l2.weight']

#retreiving the value of variable freq in this jupyter file so that words can be mapped to corresponding weights from task1
with open('hindi_freq_dictionary.txt', 'rb') as handle:
    data_dummy = handle.read()
    
freq = pickle.loads(data_dummy)

In [ ]:
#calculating the number of unique words or vocabulary size
entire_text = data['text'].values.tolist()
entire_text= [word for line in entire_text for word in line.split()]


unique_words_and_corresponding_freq = {} 
for item in entire_text: 
    if (item in unique_words_and_corresponding_freq): 
        unique_words_and_corresponding_freq[item] += 1
    else: 
        unique_words_and_corresponding_freq[item] = 1
        

no_of_unique_words_vocabulary=len(unique_words_and_corresponding_freq)


In [ ]:
#split into train and test
x_train, x_test, y_train, y_test = train_test_split(X,Y,random_state=1,test_size=0.20 )

#tokenizer that is crated for the entire vocabulary 
tokenizer = Tokenizer(num_words=no_of_unique_words_vocabulary)
tokenizer.fit_on_texts(X)
word_index=tokenizer.word_index

#fitting the tokenizor for each x_train and x_test data
x_train_tokens=tokenizer.texts_to_sequences(x_train)
x_test_tokens=tokenizer.texts_to_sequences(x_test)

In [ ]:
#to add padding based the longest sentence
#code to find the lenght of the lpngest sentence

longest_sentence=0

for each_sentence in X:
    word_list = each_sentence.split()
    number_of_words = len(word_list)
    if number_of_words > longest_sentence:
        longest_sentence=number_of_words 
        

X_train_with_padding=pad_sequences(x_train_tokens,maxlen=longest_sentence,padding='post')
X_test_with_padding=pad_sequences(x_test_tokens,maxlen=longest_sentence,padding='post')

In [ ]:
#defining the embedding matrix to use in task2
embedding_matrix_to_use=np.zeros((len(word_index)+1,300))

In [ ]:
def return_index_in_the_embedding_from_task1(word):
    return list(freq.keys()).index(word)
    
#test
#print(return_index_in_the_embedding_from_task1('देश'))
#

#this function returns the curresponing index of the the word from task1.
#the idea is to map the words in task2 to their curresponding weights in task1

In [ ]:
#each word in task2 is mapped to its currensponding weight in task1 vocabulary 
for word,index in word_index.items():
    
    
        corresponding_index_in_embedding= return_index_in_the_embedding_from_task1(word)
        numpy_corresponding= embedding_weights[corresponding_index_in_embedding].data.numpy()
        embedding_matrix_to_use[index]=numpy_corresponding
        
    
        
        
        
#print("Embedding Matrix Shape",embedding_matrix_to_use.shape)        
        

In [ ]:
# reference link- https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456

from tensorflow.keras.layers import *

embedding_layer=Embedding(len(word_index)+1,300,weights=[embedding_matrix_to_use],input_length=longest_sentence,trainable=False)

from tensorflow.keras.models import *
from tensorflow.keras.layers import *

model_keras=Sequential()

model_keras.add(embedding_layer)
model_keras.add(LSTM(units=32,dropout=0.1))
model_keras.add(Dense(1,activation="sigmoid"))

model_keras.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])

In [ ]:
model_keras.summary()

In [ ]:
print(model_keras.fit(X_train_with_padding,y_train,epochs=10,validation_data=(X_test_with_padding,y_test)))